In [ ]:
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
from surya.recognition import RecognitionPredictor
from surya.detection import DetectionPredictor

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [ ]:
langs = ["en"] # Replace with your languages or pass None (recommended to use None)
recognition_predictor = RecognitionPredictor()
detection_predictor = DetectionPredictor()

image_list = []
doc_list = []
ocr_df = pd.read_excel('ocr_df.xlsx', index_col=0)
for index, row in ocr_df.iterrows():
    directory = 'test_files'
    pdf_text = ''
    images = convert_from_path(f'''{directory}/{row['item_filename']}.pdf''')
    for i in range(len(images)):
        image_list.append(images[i])
        doc_list.append(index)

langs = [["en"] for i in range(len(image_list))]

predictions = recognition_predictor(image_list, langs, detection_predictor,
                                    recognition_batch_size=14,
                                    detection_batch_size=14)

pdf_text = [''] * len(image_list)
nl = '\n'
for index, p in enumerate(predictions):
    text_list = p.text_lines
    for text in text_list:
        if text.confidence > 0.7:
            pdf_text[doc_list[index]] = (f'{pdf_text[doc_list[index]]}{nl}{text.text}')

# save the text to the dataframe
for index, text in enumerate(pdf_text):
    ocr_df.at[index, 'surya'] = text

ocr_df.to_excel('ocr_df.xlsx')
